In [1]:
import os
import requests
import pandas as pd

* Obtención de polígono en Monte Hermoso generado por QGIS

* Sería conveniente generar un script para tomar la geometría de un shape file o de cualquier dato vectorial en proyección EPSG:4326

In [2]:
# Definición de carpeta donde se encuentra dato vectorial
vec_folder = r'/home/negrete/test_gee/Vectores/csv/'
vec_file = r'geom_monte_h.csv'
print(os.path.exists(vec_folder))
df_geom = pd.read_csv(os.path.join(vec_folder,vec_file))
display(type(df_geom.loc[0]['wkt']))
geom = df_geom.loc[0]['wkt']
# Geometría generada por QGIS en wkt
print(geom)

# Modificaciones necesarias para que funcione la búsqueda en plataforma de la ESA
geom_2 = geom.replace('MultiPolygon','POLYGON')
geom_2 = geom_2.replace('(((','((')
geom_2 = geom_2.replace(')))','))')
print(geom_2)

True


str

MultiPolygon (((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316)))
POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))


### Armado de string de búsqueda para requerir a servidor de la ESA

In [3]:
filter_Sent = "contains(Name,'MSIL2A')"
url = 'https://catalogue.dataspace.copernicus.eu/odata/v1/Products'
orderby = "ContentDate/Start"
top = "100"

initial_date_new = "gt 2022-03-03T00:00:00.000Z"
final_date_new = "lt 2022-05-03T00:00:00.000Z"
prefix_place =f"OData.CSC.Intersects(area=geography'SRID=4326;{geom_2}')"
renew_query_mis_pl = f"{url}?$filter={filter_Sent} and {prefix_place} and ContentDate/Start {initial_date_new} and ContentDate/Start {final_date_new}&$orderby={orderby}&$top={top}"
cloud_cover = str(10) # Se encuentra en procentaje pero hay que esscribirlo tipo string
prefix_cloud = f"Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le {cloud_cover})"
prefix_dates = f"ContentDate/Start {initial_date_new} and ContentDate/Start {final_date_new}"
renew_query_mis_pl_cc = f"{url}?$filter={filter_Sent} and {prefix_place} and {prefix_cloud} and {prefix_dates}&$orderby={orderby}&$top={top}"
# print(prefix_place, renew_query_mis_pl, renew_query_mis_pl_cc, sep='\n')
req_list = [renew_query_mis_pl, renew_query_mis_pl_cc]
display(req_list)

["https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'MSIL2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))') and ContentDate/Start gt 2022-03-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:00:00.000Z&$orderby=ContentDate/Start&$top=100",
 "https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'MSIL2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))') and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 10) and ContentDate/Start gt 2022-03-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:00:00.000Z&$or

In [4]:
df_list = []
for i,req in enumerate(req_list):
    # print(req)
    json = requests.get(req).json()
    display(req)
    df_list.append(pd.DataFrame.from_dict(json['value']).head(3))
    display(df_list[i])

"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'MSIL2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))') and ContentDate/Start gt 2022-03-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:00:00.000Z&$orderby=ContentDate/Start&$top=100"

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,c1253c40-3176-5322-9c70-a0a575c0e56e,S2B_MSIL2A_20220304T135109_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-04T22:22:20.540Z,2022-07-01T15:39:54.438Z,2022-07-01T15:39:54.438Z,True,,/eodata/Sentinel-2/MSI/L2A/2022/03/04/S2B_MSIL...,[],"{'Start': '2022-03-04T13:51:09.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-61.847717 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
1,application/octet-stream,0db40101-d5be-5962-8a67-935cdf44a3d6,S2B_MSIL2A_20220307T140049_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-07T23:14:20.269Z,2022-03-27T15:07:17.153Z,2022-03-27T15:07:17.153Z,True,,/eodata/Sentinel-2/MSI/L2A/2022/03/07/S2B_MSIL...,[],"{'Start': '2022-03-07T14:00:49.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-61.006195 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0061..."
2,application/octet-stream,3db1829f-7309-5c51-810f-20ffc7410946,S2A_MSIL2A_20220309T135121_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-09T21:38:02.833Z,2022-03-27T01:19:12.339Z,2022-03-27T01:19:12.339Z,True,,/eodata/Sentinel-2/MSI/L2A/2022/03/09/S2A_MSIL...,[],"{'Start': '2022-03-09T13:51:21.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-60.554108 -39.6...,"{'type': 'Polygon', 'coordinates': [[[-60.5541..."


"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'MSIL2A') and OData.CSC.Intersects(area=geography'SRID=4326;POLYGON ((-61.34108516 -38.96185316, -61.20776258 -38.96086585, -61.20776258 -38.96086585, -61.20776258 -39.02254635, -61.34172003 -39.02155988, -61.34108516 -38.96185316))') and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le 10) and ContentDate/Start gt 2022-03-03T00:00:00.000Z and ContentDate/Start lt 2022-05-03T00:00:00.000Z&$orderby=ContentDate/Start&$top=100"

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,c1253c40-3176-5322-9c70-a0a575c0e56e,S2B_MSIL2A_20220304T135109_N0400_R024_T20HPB_2...,application/octet-stream,0,2022-03-04T22:22:20.540Z,2022-07-01T15:39:54.438Z,2022-07-01T15:39:54.438Z,True,,/eodata/Sentinel-2/MSI/L2A/2022/03/04/S2B_MSIL...,[],"{'Start': '2022-03-04T13:51:09.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-61.847717 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.8477..."
1,application/octet-stream,0db40101-d5be-5962-8a67-935cdf44a3d6,S2B_MSIL2A_20220307T140049_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-07T23:14:20.269Z,2022-03-27T15:07:17.153Z,2022-03-27T15:07:17.153Z,True,,/eodata/Sentinel-2/MSI/L2A/2022/03/07/S2B_MSIL...,[],"{'Start': '2022-03-07T14:00:49.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-61.006195 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0061..."
2,application/octet-stream,c1be4316-de04-5902-b0ac-b01b32aec350,S2A_MSIL2A_20220312T140101_N0400_R067_T20HPB_2...,application/octet-stream,0,2022-03-12T23:14:08.829Z,2022-03-26T01:57:48.786Z,2022-03-26T01:57:48.786Z,True,,/eodata/Sentinel-2/MSI/L2A/2022/03/12/S2A_MSIL...,[],"{'Start': '2022-03-12T14:01:01.024Z', 'End': '...",geography'SRID=4326;POLYGON ((-61.008057 -38.8...,"{'type': 'Polygon', 'coordinates': [[[-61.0080..."
